In [16]:
import random
import time
import math
import copy


INICIO, FINAL = ('i',), ('f',)

class Formiga:
    def __init__(self, makespan=0):
        self.caminho = []
        self.makespan = makespan

class Colonia:
    def __init__(self, maquinas, tempos, qtd_trab, qtd_maq):
        self.maquinas = maquinas
        self.tempos = tempos
        self.qtd_trab = qtd_trab
        self.qtd_maq = qtd_maq
        qtd_ops = qtd_trab * qtd_maq
        self.formigas = [Formiga() for _ in range(QTD_FORMIGAS)]
        self.custos = [tempos[i // qtd_maq][i % qtd_maq] for i in range(qtd_ops)]
        self.vertices = [(i // self.qtd_maq, i % self.qtd_maq) for i in range(qtd_ops)]
        self.feromonio = [[-1 if i == j else FER_INICIAL for j in range(qtd_ops)] for i in range(qtd_ops + 1)]
        self.grafo = self.cria_grafo()
        self.desirability = self.calcula_desirability()
        
    def calcula_desirability(self):
        desirability = []
        for vertice in self.vertices:
            trab, i_maq = vertice
            tempo_finalizar = 0
            while i_maq < self.qtd_maq:
                tempo_finalizar += self.tempos[trab][i_maq]
                i_maq += 1
            desirability.append(tempo_finalizar)
        return desirability

    def cria_grafo(self):
        grafo = {}
        grafo[INICIO], grafo[FINAL]= [], []
        for i in range(self.qtd_trab):
            grafo[INICIO] += [(i, self.maquinas[i][0])]

        for i_vertice in self.vertices:
            trab, maq = i_vertice
            i_maq = self.maquinas[trab].index(maq)
            if i_maq + 1 < self.qtd_maq:
                prox_maq = self.maquinas[trab][i_maq + 1]
                grafo[i_vertice] = [(trab, prox_maq)]
            else:
                grafo[i_vertice] = [FINAL]
            for j_vertice in self.vertices:
                trab_j, maq_j = j_vertice
                if maq == maq_j and i_vertice != j_vertice:
                    grafo[i_vertice] += [(trab_j, maq_j, 'bd')]
        return grafo
    
    def grafo_direcionado(self, caminho):
        grafo_di = copy.deepcopy(self.grafo)
        for vertice in grafo_di:
            copia_adjacentes = grafo_di[vertice][:]
            for adjacente in copia_adjacentes:
                if len(adjacente) == 3:
                    adjacente_di = (adjacente[0], adjacente[1])
                    i_vertice, i_adjacente = caminho.index(vertice), caminho.index(adjacente_di)
                    if i_vertice < i_adjacente:
                        grafo_di[vertice].remove(adjacente)
                        grafo_di[vertice].append(adjacente_di)
                        grafo_di[adjacente_di].remove((vertice[0], vertice[1], 'bd'))
                    else:
                        grafo_di[adjacente_di].remove((vertice[0], vertice[1], 'bd'))
                        grafo_di[adjacente_di].append(vertice)
                        grafo_di[vertice].remove(adjacente)
        return grafo_di
    
    def index(self, trab_maq):
        if trab_maq == INICIO:
            return len(self.feromonio) - 1
        trab, maq = trab_maq
        return trab * self.qtd_maq + self.maquinas[trab].index(maq)
        
    def custo(self, adjacente):
        if adjacente == FINAL:
            return 0
        return self.custos[self.index(adjacente)]

    def calcula_makespan(self, caminho):
        grafo_di = self.grafo_direcionado(caminho)
        tempo = {vertice: -1 for vertice in caminho}
        tempo[INICIO] = tempo[FINAL] = 0
        
        for adjacente in grafo_di[INICIO]:
            tempo[adjacente] = self.custos[self.index(adjacente)]
        for vertice in caminho:
            for adjacente in grafo_di[vertice]:
                custo = self.custo(adjacente)
                if tempo[adjacente] < tempo[vertice] + custo:
                    tempo[adjacente] = tempo[vertice] + custo
        return tempo[FINAL]

    def calcula_probabilidade(self, atual, candidato):
        i_atual = self.index(atual)
        i_candidato = self.index(candidato)
        tij = self.feromonio[i_atual][i_candidato]
        nij = self.desirability[i_candidato]
        return (tij ** ALFA) * (nij ** BETA)

    def proximo(self, proximos, atual):
        somatorio_probs, probs = 0, []
        for i in range(len(proximos)):
            somatorio_probs += self.calcula_probabilidade(atual, proximos[i])
        for i in range(len(proximos)):
            probs.append(self.calcula_probabilidade(atual, proximos[i]) / somatorio_probs)

        escolhido = random.choices(proximos, weights=probs)
        trab, maq = escolhido[0]
        proximos.remove((trab, maq))
        i_maq = self.maquinas[trab].index(maq)
        if i_maq + 1 < self.qtd_maq:
            prox_maq = self.maquinas[trab][i_maq + 1]
            proximos.append((trab, prox_maq))
        return (trab, maq)

    def caminha_formiga(self, formiga):
        proximos = [(trab, self.maquinas[trab][0]) for trab in range(self.qtd_trab)]
        atual = INICIO
        while proximos:
            atual = self.proximo(proximos, atual)
            trab, maq = atual
            formiga.caminho.append((trab, maq))
        formiga.makespan = self.calcula_makespan(formiga.caminho)

    def reseta_formigas(self):
        for formiga in self.formigas:
            formiga.caminho = []
    
    def atualiza_feromonios(self, formiga):
        for i in range(len(self.feromonio)):
            for j in range(len(self.feromonio[i])):
                if self.feromonio[i][j] == -1:
                    continue
                self.feromonio[i][j] *=  1 - TX_EVA

        # for formiga in self.formigas:
        qtd_feromonio = 1 / formiga.makespan
        for i in range(len(formiga.caminho) - 1):
            i_atual = self.index(formiga.caminho[i])
            i_prox = self.index(formiga.caminho[i + 1])
            self.feromonio[i_atual][i_prox] += qtd_feromonio
        inicio = self.index(INICIO)
        primeiro = self.index(formiga.caminho[0])
        self.feromonio[inicio][primeiro] += qtd_feromonio
        self.reseta_formigas()

    def execucao(self):
        melhor_formiga = Formiga(math.inf)
        for _ in range(ITERACOES):
            for formiga in self.formigas:
                self.caminha_formiga(formiga)
                if formiga.makespan < melhor_formiga.makespan:
                    melhor_formiga.makespan = formiga.makespan
                    melhor_formiga.caminho = formiga.caminho[:]
            # print(f"{melhor_formiga.makespan} ", end='')
            self.atualiza_feromonios(melhor_formiga)
        # print(f"\nMelhor makespan: {melhor_formiga.makespan}")
        # print(f"Melhor caminho: {melhor_formiga.caminho}")
        return melhor_formiga.makespan

def arquivo_entrada(nome_arquivo):
    try:
        with open(nome_arquivo) as arquivo:
            maquinas, tempos = [], []
            linha = ""
            while not linha or not linha[0].isnumeric():
                linha = arquivo.readline().strip()
            qtd_trab, qtd_maq = [int(num) for num in linha.split()]
            for i, linha in enumerate(arquivo):
                linha_split = [int(num) for num in linha.split()]
                maquinas.append([]), tempos.append([])
                for j in range(0, len(linha_split), 2):
                    maquinas[i] += [linha_split[j]]
                    tempos[i]   += [linha_split[j + 1]]
    except FileNotFoundError:
        print("arquivo nao encontrado")
        exit(1)
    return maquinas, tempos, qtd_trab, qtd_maq

In [29]:
QTD_FORMIGAS_LIST = [25, 50, 100]
ITERACOES_LIST = [25, 50, 100]
TX_EVA_LIST = [0.1, 0.25, 0.5]
FER_INICIAL_LIST = [0.001, 0.01, 0.1]
ALFA_LIST = [0.5, 1, 2, 5]
BETA_LIST = [0.5, 1, 2, 5]

## ft06

In [18]:

ITERACOES = 25
FER_INICIAL = 0.01
TX_EVA = 0.25
ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

medias = []
for qnt_formiga in QTD_FORMIGAS_LIST:
    makespans = []
    for i in range(10):
        QTD_FORMIGAS = qnt_formiga
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Quantidade de formigas: {QTD_FORMIGAS_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Quantidade de formigas: 25: 60.8
Quantidade de formigas: 50: 59.2
Quantidade de formigas: 100: 58.7
59.2002 segundos


In [19]:
QTD_FORMIGAS = 100

FER_INICIAL = 0.01
TX_EVA = 0.25
ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

medias = []
for iteracoes in ITERACOES_LIST:
    makespans = []
    for i in range(10):
        ITERACOES = iteracoes
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Iteracoes: {ITERACOES_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Iteracoes: 25: 59.2
Iteracoes: 50: 58.6
Iteracoes: 100: 59.4
240.7951 segundos


In [23]:
QTD_FORMIGAS = 100
ITERACOES = 50

TX_EVA = 0.25
ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

medias = []
for fer_inicial in FER_INICIAL_LIST:
    makespans = []
    for i in range(10):
        FER_INICIAL = fer_inicial
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Feromonio inicial: {FER_INICIAL_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Feromonio inicial: 0.001: 60.0
Feromonio inicial: 0.01: 59.3
Feromonio inicial: 0.1: 57.7
Feromonio inicial: 0.25: 57.7
277.2302 segundos


In [25]:
QTD_FORMIGAS = 100
ITERACOES = 50
FER_INICIAL = 0.1

ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

medias = []
for tx_eva in TX_EVA_LIST:
    makespans = []
    for i in range(10):
        TX_EVA = tx_eva
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Taxa evaporacao: {TX_EVA_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Taxa evaporacao: 0.1: 58.7
Taxa evaporacao: 0.25: 59.3
Taxa evaporacao: 0.5: 58.7
204.9405 segundos


In [26]:
QTD_FORMIGAS = 100
ITERACOES = 50
FER_INICIAL = 0.1
TX_EVA = 0.1

BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

medias = []
for alfa in ALFA_LIST:
    makespans = []
    for i in range(10):
        ALFA = alfa
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Alfa: {ALFA_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Alfa: 0.5: 58.9
Alfa: 1: 58.2
Alfa: 2: 58.8
Alfa: 5: 58.5
277.1430 segundos


In [27]:
QTD_FORMIGAS = 100
ITERACOES = 50
FER_INICIAL = 0.1
TX_EVA = 0.1
ALFA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

medias = []
for beta in BETA_LIST:
    makespans = []
    for i in range(10):
        BETA = beta
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Beta: {BETA_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Beta: 0.5: 58.0
Beta: 1: 58.1
Beta: 2: 55.8
Beta: 5: 56.5
269.7323 segundos


In [28]:
## MELHOR FT06
## QTD_FORMIGAS = 100
## ITERACOES = 50
## FER_INICIAL = 0.1
## TX_EVA = 0.1
## ALFA = 1
## BETA = 2

## la01

In [30]:

ITERACOES = 25
FER_INICIAL = 0.01
TX_EVA = 0.25
ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

medias = []
for qnt_formiga in QTD_FORMIGAS_LIST:
    makespans = []
    for i in range(10):
        QTD_FORMIGAS = qnt_formiga
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Quantidade de formigas: {QTD_FORMIGAS_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Quantidade de formigas: 25: 742.0
Quantidade de formigas: 50: 729.1
Quantidade de formigas: 100: 719.2
134.3801 segundos


In [31]:
QTD_FORMIGAS = 100

FER_INICIAL = 0.01
TX_EVA = 0.25
ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

medias = []
for iteracoes in ITERACOES_LIST:
    makespans = []
    for i in range(10):
        ITERACOES = iteracoes
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Iteracoes: {ITERACOES_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Iteracoes: 25: 719.3
Iteracoes: 50: 723.0
Iteracoes: 100: 723.0
517.3064 segundos


In [32]:
QTD_FORMIGAS = 100
ITERACOES = 25

TX_EVA = 0.25
ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

medias = []
for fer_inicial in FER_INICIAL_LIST:
    makespans = []
    for i in range(10):
        FER_INICIAL = fer_inicial
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Feromonio inicial: {FER_INICIAL_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Feromonio inicial: 0.001: 715.5
Feromonio inicial: 0.01: 709.6
Feromonio inicial: 0.1: 728.2
222.3795 segundos


In [34]:
QTD_FORMIGAS = 100
ITERACOES = 25
FER_INICIAL = 0.01

ALFA = 1
BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

medias = []
for tx_eva in TX_EVA_LIST:
    makespans = []
    for i in range(10):
        TX_EVA = tx_eva
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Taxa evaporacao: {TX_EVA_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Taxa evaporacao: 0.1: 732.3
Taxa evaporacao: 0.25: 722.2
Taxa evaporacao: 0.5: 734.1
216.3741 segundos


In [35]:
QTD_FORMIGAS = 100
ITERACOES = 25
FER_INICIAL = 0.01
TX_EVA = 0.25

BETA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

medias = []
for alfa in ALFA_LIST:
    makespans = []
    for i in range(10):
        ALFA = alfa
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Alfa: {ALFA_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Alfa: 0.5: 738.9
Alfa: 1: 713.0
Alfa: 2: 729.4
Alfa: 5: 735.6
302.3548 segundos


In [36]:
QTD_FORMIGAS = 100
ITERACOES = 25
FER_INICIAL = 0.01
TX_EVA = 0.25
ALFA = 1

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

medias = []
for beta in BETA_LIST:
    makespans = []
    for i in range(10):
        BETA = beta
        colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
        makespans.append(colonia.execucao())
    medias.append(sum(makespans) / len(makespans))
    
for i in range(len(medias)):
    print(f"Beta: {BETA_LIST[i]}: {medias[i]}")
print(f"{time.time() - start:.4f} segundos")

Beta: 0.5: 735.6
Beta: 1: 725.3
Beta: 2: 685.9
Beta: 5: 673.8
297.0101 segundos


In [ ]:
## MELHOR LA01
## QTD_FORMIGAS = 100
## ITERACOES = 25
## FER_INICIAL = 0.01
## TX_EVA = 0.25
## ALFA = 1
## BETA = 5

In [44]:
import numpy as np
import math

## Usando parametros do ft06

In [50]:
QTD_FORMIGAS = 100
ITERACOES = 50
FER_INICIAL = 0.1
TX_EVA = 0.1
ALFA = 1
BETA = 2

In [46]:
#ft06

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

otimo = 55
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 56.7
Desvio padrão: 1.4866068747318504
Diferenca otimo: 0
Num otimos: 7
138.3942 segundos


In [49]:
#la01

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

otimo = 666
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 706.45
Desvio padrão: 17.168211904563623
Diferenca otimo: 2
Num otimos: 0
305.5543 segundos


In [51]:
#la29

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la29.txt")

otimo = 1157
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 1706.9
Desvio padrão: 30.54488500551279
Diferenca otimo: 509
Num otimos: 0
3146.2992 segundos


In [52]:
#la40

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la40.txt")

otimo = 1222
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 1825.95
Desvio padrão: 40.35898289104917
Diferenca otimo: 529
Num otimos: 0
2802.6885 segundos


## Usando parametros do la01

In [53]:
QTD_FORMIGAS = 100
ITERACOES = 25
FER_INICIAL = 0.01
TX_EVA = 0.25
ALFA = 1
BETA = 5

In [54]:
#ft06

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/ft06.txt")

otimo = 55
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 57.1
Desvio padrão: 1.044030650891055
Diferenca otimo: 0
Num otimos: 2
68.8270 segundos


In [55]:
#la01

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la01.txt")

otimo = 666
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 676.7
Desvio padrão: 10.663489110042736
Diferenca otimo: 0
Num otimos: 7
151.9092 segundos


In [56]:
#la29

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la29.txt")

otimo = 1157
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 1659.25
Desvio padrão: 17.03489066592445
Diferenca otimo: 471
Num otimos: 0
1585.1327 segundos


In [57]:
#la40

start = time.time()
maquinas, tempos, qtd_trab, qtd_maq = arquivo_entrada(f"files/la40.txt")

otimo = 1222
num_otimos = media = desvio_padrao = 0
makespans = []
melhor_makespan = math.inf
for i in range(20):
    colonia = Colonia(maquinas, tempos, qtd_trab, qtd_maq)
    makespan = colonia.execucao()
    if makespan < melhor_makespan:
        melhor_makespan = makespan
    if makespan == otimo:
        num_otimos += 1
    makespans.append(makespan)

media = np.mean(makespans)
desvio_padrao = np.std(makespans)    
print(f"Media: {media}\nDesvio padrão: {desvio_padrao}\nDiferenca otimo: {melhor_makespan - otimo}\nNum otimos: {num_otimos}")
print(f"{time.time() - start:.4f} segundos")

Media: 1809.95
Desvio padrão: 26.72728007111835
Diferenca otimo: 523
Num otimos: 0
1347.6169 segundos
